### Spark Context
- spark app 과 cluster 연결 관리하는 객체

In [52]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._

val conf = new SparkConf().setMaster("local").setAppName("RddSample")
val sc1 = new SparkContext(conf)

lastException = null


Name: org.apache.spark.SparkException
Message: Only one SparkContext may be running in this JVM (see SPARK-2243). To ignore this error, set spark.driver.allowMultipleContexts = true. The currently running SparkContext was created at:
org.apache.spark.sql.SparkSession$Builder.getOrCreate(SparkSession.scala:860)
org.apache.toree.kernel.api.Kernel.sparkSession(Kernel.scala:425)
org.apache.toree.kernel.api.Kernel.sparkContext(Kernel.scala:429)
$line8.$read$$iw$$iw$$iw$$iw.sc(<console>:17)
$line13.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(<console>:27)
$line13.$read$$iw$$iw$$iw$$iw$$iw.<init>(<console>:40)
$line13.$read$$iw$$iw$$iw$$iw.<init>(<console>:42)
$line13.$read$$iw$$iw$$iw.<init>(<console>:44)
$line13.$read$$iw$$iw.<init>(<console>:46)
$line13.$read$$iw.<init>(<console>:48)
$line13.$read.<init>(<console>:50)
$line13.$read$.<init>(<console>:54)
$line13.$read$.<clinit>(<console>)
$line13.$eval$.$print$lzycompute(<console>:7)
$line13.$eval$.$print(<console>:6)
$line13.$eval.$print(<console

#### 2.1.3 RDD 생성

- 드라이버 프로그램의 collection object 이용하는 방법

In [3]:
val rdd1 = sc1.parallelize(List("a", "b", "c", "d"))

rdd1 = ParallelCollectionRDD[0] at parallelize at <console>:33


ParallelCollectionRDD[0] at parallelize at <console>:33

- 파일이나 데이터 데이스 같은 외부 데이터를 읽어서 새로운 RDD 생성하는 방법

    - textFile의 파일 각줄은 한 개의 RDD 구성요소가 됨

In [4]:
val rdd2 = sc1.textFile("RDD.ipynb")

rdd2 = RDD.ipynb MapPartitionsRDD[2] at textFile at <console>:33


RDD.ipynb MapPartitionsRDD[2] at textFile at <console>:33

### 2.1.4 RDD 기본 액션

#### 2.1.4.1 collect

- RDD의 모든 원소를 모아서 배열로 돌려줍니다.
- collect 연산을 수행하면 RDD에 있는 다른 모든 요소들이 collect 연산을 호출한 서버의 메모리에 수집되기 때문에 전체 데이터를 담을 수 있는 충분한 메모리 공간이 확보되었을 때 사용

In [11]:
val rdd = sc1.parallelize(1 to 10)
val result = rdd.collect
println(result.mkString(", "))

1, 2, 3, 4, 5, 6, 7, 8, 9, 10


rdd = ParallelCollectionRDD[9] at parallelize at <console>:36
result = Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

#### 2.1.4.2 count

* count는 RDD구성하는 전체 요소의 개수를 반환

In [12]:
val rdd = sc1.parallelize(1 to 10)
val result = rdd.count
println(result)

10


rdd = ParallelCollectionRDD[10] at parallelize at <console>:36
result = 10


10

### 2.1.5 RDD transformation

- 기존 RDD를 이용해 새로운 RDD를 생성하는 연산
- 분류
    - 맵 : 요소 간의 사상을 정의한 함수를 RDD에 속하는 모든 요소에 적용해 새로운 RDD 생성
    - 그룹화 : 특정 조건에 따라 요소를 그룹화하거나 특정함수를 적용
    - 집합 : RDD에 포함된 요소를 하나의 집합으로 간주할 때 서로 다른 RDD 간에 합집합, 교집합 등을 계산
    - 파티션 : RDD의 파티션 개수를 조정
    - 필터와 정렬 연산 : 특정 조건을 만족하는 요소만 선택하거나 각 요소를 정해진 기준에 따라 정렬
- RDD가 제공하는 주요 연산이 RDD를 구성하고 있는 데이터 유형과 밀접한 관계를 맺는다

#### [map 관련 연산]

##### map
map은 하나의 입력을 받아서 하나의 값을 돌려주는 함수를 인자로 받습니다.
이 함수를 RDD에 속하는 모든 요소에 적용한 뒤 그 결과로 구성된 새로운 RDD를 생성해 반환합니다.

In [14]:
val rdd = sc1.parallelize(1 to 10)
val result = rdd.map(_ + 1)
println(result.collect.mkString(", "))

2, 3, 4, 5, 6, 7, 8, 9, 10, 11


rdd = ParallelCollectionRDD[13] at parallelize at <console>:36
result = MapPartitionsRDD[14] at map at <console>:37


MapPartitionsRDD[14] at map at <console>:37

** RDD는 그 자체로는 타입이 될 수 없고 반드시 타입 매개변수를 지정해서 정의해야 하는 매개변수화한 타입

`map[U](f: (T) => U)(implicit arg0: ClassTag[U]): RDD[U]`

`T타입을 U타입으로 변환하는 함수 f를 이용해 RDD<T> 타입의 RDD를 RDD<U> 타입으로 변환하는 메서드`

#### 2.1.5.2 flatMap

- 각 배열 속에 포함된 요소를 모두 배열 밖으로 끄집어 내는 작어을 해야 할 경우
- 하나의 입력값에 대응하는 반환값이 여러애리 경우

In [16]:
// 단어 3개를 가진 List 생성
val fruits = List("apple,orange", "grape,apple,mango", "blueberry,tomato,orange")

// RDD 생성
val rdd1 = sc.parallelize(fruits)

// RDD의 map() 메서드를 각 단어를 ","를 기준으로 분리
val rdd2 = rdd1.map(_.split(","))

println(rdd2.collect().map(_.mkString("{", ", ", "}")).mkString("{", ", ", "}"))

{{apple, orange}, {grape, apple, mango}, {blueberry, tomato, orange}}


fruits = List(apple,orange, grape,apple,mango, blueberry,tomato,orange)
rdd1 = ParallelCollectionRDD[17] at parallelize at <console>:40
rdd2 = MapPartitionsRDD[18] at map at <console>:43


MapPartitionsRDD[18] at map at <console>:43

In [18]:
val fruits = List("apple,orange", "grape,apple,mango", "blueberry,tomato,orange")
val rdd1 = sc.parallelize(fruits)
val rdd2 = rdd1.flatMap(_.split(","))

println(rdd2.collect().mkString("{", ", ", "}"))

{apple, orange, grape, apple, mango, blueberry, tomato, orange}


fruits = List(apple,orange, grape,apple,mango, blueberry,tomato,orange)
rdd1 = ParallelCollectionRDD[19] at parallelize at <console>:37
rdd2 = MapPartitionsRDD[20] at flatMap at <console>:38


MapPartitionsRDD[20] at flatMap at <console>:38

In [21]:
val fruits = List("apple,orange", "grape,apple,mango", "blueberry,tomato,orange")
val rdd1 = sc.parallelize(fruits)
val rdd2 = rdd1.flatMap(log => {
    // apple이라는 단어가 포함된 경우만 처리하고 싶다.
    if (log.contains("apple")) {
        Some(log.indexOf("apple"))
    } else {
        None
    }
})
println(rdd2.collect().mkString("{", ", ", "}"))

{0, 6}


fruits = List(apple,orange, grape,apple,mango, blueberry,tomato,orange)
rdd1 = ParallelCollectionRDD[23] at parallelize at <console>:37
rdd2 = MapPartitionsRDD[24] at flatMap at <console>:38


MapPartitionsRDD[24] at flatMap at <console>:38

#### mapPartitions

map, flatMap RDD의 각 요소를 하나씩 처리한다면
mapPartitions()는 파티션 단위로 처리
인자로 전달받은 함수를 파티션 단위로 적용하고 그 결과로 구성된 새로운 RDD를 생성하는 메서드

파티션 단위로 파티션에 속한 모든 요소를 한번의 함수 호출로 처리할 수 있음

데이터베이스 연결과 같은 고비용의 자원을 파티션 단위로 공유해 사용할 수 있는 장점

In [22]:
val rdd1 = sc.parallelize(1 to 10, 3)
val rdd2 = rdd1.mapPartitions(numbers => {
    println("DB 연결")
    numbers.map{
        number => number + 1
    }
})
println(rdd2.collect().mkString( ", "))

2, 3, 4, 5, 6, 7, 8, 9, 10, 11


rdd1 = ParallelCollectionRDD[25] at parallelize at <console>:36
rdd2 = MapPartitionsRDD[26] at mapPartitions at <console>:37


MapPartitionsRDD[26] at mapPartitions at <console>:37

#### 2.1.5.4 mapPartitionWithIndex

인자로 전달받은 함수를 파티션 단위로 적용하고 그 결과값으로 구성된 새로운 RDDf르 생성하는 메서드

mapPartition과 다른 점은 파티션의 인덱스 정보도 함께 전달해 준다는 점

In [23]:
val rdd1 = sc.parallelize(1 to 10, 3)
val rdd2 = rdd1.mapPartitionsWithIndex((idx, numbers) => {
    numbers.flatMap {
        case number if idx == 1 => Option(number + 1)
        case _                  => None
    }
})
println(rdd2.collect().mkString( ", "))

5, 6, 7


rdd1 = ParallelCollectionRDD[27] at parallelize at <console>:35
rdd2 = MapPartitionsRDD[28] at mapPartitionsWithIndex at <console>:36


MapPartitionsRDD[28] at mapPartitionsWithIndex at <console>:36

#### mapValues

RDD의 요소가 키와 값이 쌍으로 이루고 있는 경우 PairRDD라는 용어를 사용

- RDD의 모든 요소들이 키와 값의 쌍을 이루고 있는 경우에만 사용 가능한 메서드
- 인자로 전달받은 함수를 "값"에 해당하는 요소에만 적용하고 그 결과로 구성된 새로운 RDD를 생성

In [25]:
// (키,값) 쌍으로 구성된 RDD 생성
val rdd1 = sc.parallelize(List("a", "b", "c")).map((_, 1))
val result = rdd1.mapValues(_ + 1)
println(result.collect.mkString("\t"))

(a,2)	(b,2)	(c,2)


rdd1 = MapPartitionsRDD[30] at map at <console>:41
result = MapPartitionsRDD[31] at mapValues at <console>:42


MapPartitionsRDD[31] at mapValues at <console>:42

#### 2.1.5.6 flatMapValues

- RDD의 구성요소가 키와 값의 쌍으로 구성된 경우에만 적용
- 키는 그대로 두고 값에 해당하는 요소만을 대상으로 flatMap() 연산을 적용

In [27]:
val rdd = sc.parallelize(Seq((1, "a,b"), (2, "a,c"), (3, "d,e")))
val result = rdd.flatMapValues(_.split(","))
println(result.collect.mkString("\t"))

(1,a)	(1,b)	(2,a)	(2,c)	(3,d)	(3,e)


rdd = ParallelCollectionRDD[34] at parallelize at <console>:35
result = MapPartitionsRDD[35] at flatMapValues at <console>:36


MapPartitionsRDD[35] at flatMapValues at <console>:36

### [그룹과 관련된 연산들]

#### 2.1.5.7 zip

- 두개의 서로 다른 RDD를 각 요소의 인덱스에 따라 하나의 (키, 값) 쌍으로 묶어줍니다.
- 첫번째 RDD의 n번째 요소를 키로하고 두번째 RDD의 n번째 요소를 값으로 하는 순서쌍을 생성
- 이때 두 RDD는 같은 개수의 파티션을 가지고 있고 각 파티션에 속하는 요소의 수는 동일하다고 가정
- 서로 크기가 다른 RDD 간에는 zip() 메서드를 사용할 수 없습니다.

In [28]:
val rdd1 = sc.parallelize(List("a", "b", "c"))
val rdd2 = sc.parallelize(List(1,2,3))
val result = rdd1.zip(rdd2)
println(result.collect.mkString(", "))

(a,1), (b,2), (c,3)


rdd1 = ParallelCollectionRDD[36] at parallelize at <console>:38
rdd2 = ParallelCollectionRDD[37] at parallelize at <console>:39
result = ZippedPartitionsRDD2[38] at zip at <console>:40


ZippedPartitionsRDD2[38] at zip at <console>:40

#### 2.1.5.8 zipPartitions

- 파티션 단위로 zip() 연산을 수행하고 특정함수를 적용해 그 결과로 구성된 새로운 RDD를 생성
- RDD의 파티션 개수만 동일하면 됨
- zipPartitions는 최대 4개의 RDD 지정 가능
- 병합에 사용할 함수를 인자로 전달받아서 사용할 수 있음

In [31]:
val rdd1 = sc.parallelize(List("a", "b", "c"), 3)
val rdd2 = sc.parallelize(List(1,2,3,5,6), 3)
val result = rdd1.zipPartitions(rdd2) {
    (it1, it2) => for {
        v1 <- it1
        v2 <- it2
    } yield v1 + v2
}

println(result.collect.mkString(", "))

a1, b2, b3, c5, c6


rdd1 = ParallelCollectionRDD[42] at parallelize at <console>:36
rdd2 = ParallelCollectionRDD[43] at parallelize at <console>:37
result = ZippedPartitionsRDD2[44] at zipPartitions at <console>:38


ZippedPartitionsRDD2[44] at zipPartitions at <console>:38

#### 2.1.5.9 groupBy

- RDD의 요소를 일정한 기준에 따라 여러개의 그룹으로 나누고 이 그룹으로 구성된 새로운 RDD를 생성
- 각 그룹은 키와 그 키에 속한 요소의 시퀀스로 구성
- 메서드의 인자로 전달하는 함수가 각 그룹의 키를 결정하는 역할을 담당

In [36]:
val rdd = sc.parallelize(1 to 10)
val result = rdd.groupBy{
    case i: Int if (i % 2 == 0) => "even"
    case i: Int if (i % 2 == 1) => "odd"
    case _                      => ""
}
result.collect.foreach {
    v => println(s"""${v._1}, [${v._2.mkString(",")}]""")
}

even, [2,4,6,8,10]
odd, [1,3,5,7,9]


rdd = ParallelCollectionRDD[46] at parallelize at <console>:38
result = ShuffledRDD[48] at groupBy at <console>:39


ShuffledRDD[48] at groupBy at <console>:39

#### 2.1.5.10 groupByKey

- RDD의 구성요소가 키와 값으로 쌍으로 이루어진 경우에 사용 가능
- 키를 기준으로 같은 키를 가진 요소들로 그룹을 만들고 이 그룹들로 구성된 새로운 RDD를 생성

In [37]:
val rdd = sc.parallelize(List("a", "b", "c", "b", "c")).map((_, 1))
val result = rdd.groupByKey
result.collect.foreach {
    v => println(s"""${v._1}, [${v._2.mkString(",")}]""")
}

a, [1]
b, [1,1]
c, [1,1]


rdd = MapPartitionsRDD[50] at map at <console>:35
result = ShuffledRDD[51] at groupByKey at <console>:36


ShuffledRDD[51] at groupByKey at <console>:36

#### 2.1.5.11 cogroup

- RDD의 구성요소가 키와 값의 쌍으로 구성된 경우에만 사용할 수 있는 메서드
- 여러 RDD에서 같은 키를 갖는 값 요소를 찾아서 키와 그 키에 속하는 요소의 시퀀스로 구성된 튜플을 만들고 그 튜플들로 구성된 새로운 RDD를 생성

In [40]:
val rdd1 = sc.parallelize(List(("k1", "v1"), ("k2", "v2"), ("k3", "v3"), ("k3", "v4")))
val rdd2 = sc.parallelize(List(("k1", "v5")))
val result = rdd1.cogroup(rdd2)
result.collect.foreach {
    case (k, (v_1, v_2)) => {
        println(s"""($k, [${v_1.mkString(",")}], [${v_2.mkString(",")}])""")
    }
}

(k3, [v3,v4], [])
(k2, [v2], [])
(k1, [v1], [v5])


rdd1 = ParallelCollectionRDD[56] at parallelize at <console>:36
rdd2 = ParallelCollectionRDD[57] at parallelize at <console>:37
result = MapPartitionsRDD[59] at cogroup at <console>:38


MapPartitionsRDD[59] at cogroup at <console>:38

#### 2.1.5.12 distint

- RDD의 원소에서 중복을 제외한 요소로만 구성된 새로운 RDD를 생성하는 메서드

In [41]:
val rdd = sc.parallelize(List(1,2,3,4,5,5,6,2))
val result = rdd.distinct
println(result.collect.mkString(", "))

4, 1, 6, 3, 5, 2


rdd = ParallelCollectionRDD[60] at parallelize at <console>:38
result = MapPartitionsRDD[63] at distinct at <console>:39


MapPartitionsRDD[63] at distinct at <console>:39

#### 2.1.5.13 cartesian

- 두 RDD 요소의 카테시안곱을 구하고 그 결과를 요소로 하는 새로운 RDD를 생성

In [42]:
val rdd1 = sc.parallelize(List("a", "b", "c"))
val rdd2 = sc.parallelize(List(1,2,3))
val result = rdd1.cartesian(rdd2)
println(result.collect.mkString(", "))

(a,1), (a,2), (a,3), (b,1), (b,2), (b,3), (c,1), (c,2), (c,3)


rdd1 = ParallelCollectionRDD[64] at parallelize at <console>:38
rdd2 = ParallelCollectionRDD[65] at parallelize at <console>:39
result = CartesianRDD[66] at cartesian at <console>:40


CartesianRDD[66] at cartesian at <console>:40

##### 2.1.5.14 subtract

- rdd1과 rdd2라는 두개의 RDD가 있을 때, rdd1.subtract(rdd2)는 rdd1에는 속하고 rdd2엔느 속하지 않는 요소로 구성된 새로운 RDD 생성

In [1]:
val rdd1 = sc.parallelize(List("a", "b", "c"))
val rdd2 = sc.parallelize(List("a","c"))
val result = rdd1.subtract(rdd2)
println(result.collect.mkString(", "))

b


rdd1 = ParallelCollectionRDD[0] at parallelize at <console>:27
rdd2 = ParallelCollectionRDD[1] at parallelize at <console>:28
result = MapPartitionsRDD[5] at subtract at <console>:29


MapPartitionsRDD[5] at subtract at <console>:29

#### 2.1.5.15  union

- rdd1과 rdd2라는 두개의 RDD가 있을때 rdd1 또는 rdd2에 속하는 요소로 구성된 새로운 RDD를 생성하는 메서드

In [2]:
val rdd1 = sc.parallelize(List("a", "b", "c"))
val rdd2 = sc.parallelize(List("d", "e", "f", "a"))
val result = rdd1.union(rdd2)
println(result.collect.mkString(", "))

a, b, c, d, e, f, a


rdd1 = ParallelCollectionRDD[6] at parallelize at <console>:31
rdd2 = ParallelCollectionRDD[7] at parallelize at <console>:32
result = UnionRDD[8] at union at <console>:33


UnionRDD[8] at union at <console>:33

#### 2.1.5.16 intersection

- rdd1과 rdd2라는 두개의 RDD가 있을 때 rdd1과 rdd2에 동시에 속하는 요소로 구성된 새로운 RDD를 생성하는 메서드

In [6]:
val rdd1 = sc.parallelize(List("a", "b", "c","e","a", "b"))
val rdd2 = sc.parallelize(List("d", "e", "f", "a"))
val result = rdd1.intersection(rdd2)
println(result.collect.mkString(", "))

a, e


rdd1 = ParallelCollectionRDD[33] at parallelize at <console>:31
rdd2 = ParallelCollectionRDD[34] at parallelize at <console>:32
result = MapPartitionsRDD[40] at intersection at <console>:33


MapPartitionsRDD[40] at intersection at <console>:33

#### 2.1.5.17 join

- RDD 의 구성요소가 키와 값의 쌍으로 구성된 경우에 사용할 수 있는 메서드 입니다.
- 두 RDD에서 같은 키를 가지고 있는 요소를 모아서 그룹을 형성하고, 그 결과로 구성된 새로운 RDD를 생성하는 메서드

In [7]:
val rdd1 = sc.parallelize(List("a", "b", "c","e","a", "b")).map((_, 1))
val rdd2 = sc.parallelize(List("d", "e", "f", "a")).map((_, 2))
val result = rdd1.join(rdd2)
println(result.collect.mkString("\n"))

(a,(1,2))
(a,(1,2))
(e,(1,2))


rdd1 = MapPartitionsRDD[42] at map at <console>:31
rdd2 = MapPartitionsRDD[44] at map at <console>:32
result = MapPartitionsRDD[47] at join at <console>:33


MapPartitionsRDD[47] at join at <console>:33

#### 2.1.5.18 leftOIuterJoin, rightOuterJoin

In [8]:
val rdd1 = sc.parallelize(List("a", "b", "c","e","a", "b")).map((_, 1))
val rdd2 = sc.parallelize(List("d", "e", "f", "a")).map((_, 2))
val result = rdd1.leftOuterJoin(rdd2)
println(result.collect.mkString("\n"))

(a,(1,Some(2)))
(a,(1,Some(2)))
(b,(1,None))
(b,(1,None))
(c,(1,None))
(e,(1,Some(2)))


rdd1 = MapPartitionsRDD[49] at map at <console>:31
rdd2 = MapPartitionsRDD[51] at map at <console>:32
result = MapPartitionsRDD[54] at leftOuterJoin at <console>:33


MapPartitionsRDD[54] at leftOuterJoin at <console>:33

In [9]:
val rdd1 = sc.parallelize(List("a", "b", "c","e","a", "b")).map((_, 1))
val rdd2 = sc.parallelize(List("d", "e", "f", "a")).map((_, 2))
val result = rdd1.rightOuterJoin(rdd2)
println(result.collect.mkString("\n"))

(a,(Some(1),2))
(a,(Some(1),2))
(d,(None,2))
(e,(Some(1),2))
(f,(None,2))


rdd1 = MapPartitionsRDD[56] at map at <console>:31
rdd2 = MapPartitionsRDD[58] at map at <console>:32
result = MapPartitionsRDD[61] at rightOuterJoin at <console>:33


MapPartitionsRDD[61] at rightOuterJoin at <console>:33

#### 2.1.5.19 subtractByKey

- RDD의 구성요소가 키와 값의 쌍으로 구성된 경우
- rdd1과 rdd2라는 두 RDD가 있을 때 rdd1.subtractByKey(rdd2)는 rdd1의 요소중에서 rdd2에 같은 키가 존재하는 요소를 제외

In [10]:
val rdd1 = sc.parallelize(List("a", "b", "c","e","a", "b")).map((_, 1))
val rdd2 = sc.parallelize(List("d", "e", "f", "a")).map((_, 2))
val result = rdd1.subtractByKey(rdd2)
println(result.collect.mkString("\n"))

(b,1)
(b,1)
(c,1)


rdd1 = MapPartitionsRDD[63] at map at <console>:31
rdd2 = MapPartitionsRDD[65] at map at <console>:32
result = SubtractedRDD[66] at subtractByKey at <console>:33


SubtractedRDD[66] at subtractByKey at <console>:33

### [집계와 관련된 연산들]

#### 2.1.5.20 reduceByKey

- RDD의 구성요소가 키와 값의 쌍으로 구성된 경우
- 같은 키를 가진 값들을 하나로 병합해 키-값 쌍으로 구성된 새로운 RDD를 생성

In [11]:
val rdd = sc.parallelize(List("a", "b", "a")).map((_, 1))
val result = rdd.reduceByKey(_ + _)
println(result.collect.mkString(","))

(a,2),(b,1)


rdd = MapPartitionsRDD[68] at map at <console>:31
result = ShuffledRDD[69] at reduceByKey at <console>:32


ShuffledRDD[69] at reduceByKey at <console>:32

#### 2.1.5.21 foldByKey

- RDD의 구성요소가 키와 값의 쌍으로 구성된 경우
- 전반적인 동작은 reduceByKey 유사
- 병합 연산의 초기 값을 메서더의 인자로 전달해서 병합시 사용할 수 있다는 점에서 차이

In [13]:
val rdd = sc.parallelize(List("a", "b", "a")).map((_, 1))
val result = rdd.foldByKey(1)(_ + _)
println(result.collect.mkString(","))

(a,4),(b,2)


rdd = MapPartitionsRDD[74] at map at <console>:30
result = ShuffledRDD[75] at foldByKey at <console>:31


ShuffledRDD[75] at foldByKey at <console>:31

#### 2.1.5.22 combineByKey

- RDD의 구성요소가 키와 값의 쌍으로 구성된 경우
- 같은 키를 가진 값들을 하나로 병합하는 기능을 수행하지만 병합을 수행하는 과정에서 값의 타입이 바뀔 수 있다.
    - createCombiner : 값을 병합하기 위한 콤바이너
    - mergeValue : 키에 대한 콤바이너가 이미 존재한담녀 새로운 콤바이너를 만들지 않고 병합
    - mergeCombiner : 
        - createCombiner(), mergeValue()는 파티션 단위
        - mergeCombiner는 병합이 끝난 콤바이너들끼리 다시 병합

In [18]:
case class Record(var amount: Long, var number: Long = 1) {
    def map(v: Long) = Record(v)
    def add(amount: Long): Record = {
        add(map(amount))
    }
    def add(other: Record): Record = {
        this.number += other.number
        this.amount += other.amount
        this
    }
    override def toString: String = s"avg: ${amount / number}"
}

// combineByKey()를 이용한 평균값 계산
val data = Seq(("A", 100L), ("B", 80L), ("A", 50L), ("B", 60L), ("B", 90L))
val rdd = sc.parallelize(data)
val createCombiner = (v: Long) => Record(v)
val mergeValue = (c: Record, v: Long) => c.add(v)
val mergeCombiners = (c1: Record, c2: Record) => c1.add(c2)
val result = rdd.combineByKey(createCombiner, mergeValue, mergeCombiners)
println(result.collect.mkString("\n"))

(A,avg: 75)
(B,avg: 76)


defined class Record
data = List((A,100), (B,80), (A,50), (B,60), (B,90))
rdd = ParallelCollectionRDD[84] at parallelize at <console>:39
createCombiner = > Record = <function1>
mergeValue = > Record = <function2>
mergeCombiners = > Record = <function2>
result = ShuffledRDD[85] at combineByKey at <console>:43


lastException: Throwable = null


ShuffledRDD[85] at combineByKey at <console>:43

#### 2.1.5.23 aggregateByKey

- RDD의 구성요소가 키와 값의 쌍으로 구성된 경우
- 병합을 시작할 때 초깃값을 생성하는 하는 부분을 제외하면 combineByKey와 동일한 동작을 수행

In [20]:
// combineByKey()를 이용한 평균값 계산
val data = Seq(("A", 100L), ("B", 80L), ("A", 50L), ("B", 70L), ("B", 90L))
val rdd = sc.parallelize(data)

val zero = Record(0, 0)
val mergeValue = (c: Record, v: Long) => c.add(v)
val mergeCombiners = (c1: Record, c2: Record) => c1.add(c2)

val result = rdd.aggregateByKey(zero)(mergeValue, mergeCombiners)
println(result.collect.mkString("\n"))

(A,avg: 75)
(B,avg: 80)


Name: java.lang.ArithmeticException
Message: / by zero
StackTrace:   at Record.toString(<console>:34)
  at scala.runtime.ScalaRunTime$.scala$runtime$ScalaRunTime$$inner$1(ScalaRunTime.scala:332)
  at scala.runtime.ScalaRunTime$.stringOf(ScalaRunTime.scala:337)
  at scala.runtime.ScalaRunTime$.replStringOf(ScalaRunTime.scala:345)
  at .$print$lzycompute(<console>:12)
  at .$print(<console>:6)
  at $print(<console>)
  at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
  at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
  at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
  at java.lang.reflect.Method.invoke(Method.java:498)
  at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:786)
  at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1047)
  at scala.tools.nsc.interpreter.IMain$WrappedRequest$$anonfun$loadAndRunReq$1.apply(IMain.scala:638)
  at scala.tools.nsc.interpreter.IMa

### pipe 및 파티션과 관련된 연산

#### 2.1.5.24 pipe

- pipe를 이용하면 데이터를 처리하는 과정에서 외부 프로세스를 활용할 수 있음

In [22]:
val rdd = sc.parallelize(List("1,2,3", "4,5,6", "7,8,9"))
val result = rdd.pipe("cut -f 1,3 -d ,")
println(result.collect.mkString(", "))

1,3, 4,6, 7,9


rdd = ParallelCollectionRDD[92] at parallelize at <console>:36
result = PipedRDD[93] at pipe at <console>:37


lastException: Throwable = null


PipedRDD[93] at pipe at <console>:37

#### 2.1.5.25 coalesce 와 repartition

- 현재 RDD의 파티션 개수를 조정
- repartion()
    - 파티션 수를 줄이거나 늘이거나 가능
    - 셔플기반으로 동작 수행
- coalesce()
    - 파티션을 줄이는 것만 가능
    - 셔플을 사용하지 않고

In [24]:
val rdd1 = sc.parallelize(1 to 1000000, 10)
val rdd2 = rdd1.coalesce(5)
val rdd3 = rdd2.repartition(10)

println(s"partition size: ${rdd1.getNumPartitions}")
println(s"partition size: ${rdd2.getNumPartitions}")
println(s"partition size: ${rdd3.getNumPartitions}")

partition size: 10
partition size: 5
partition size: 10


rdd1 = ParallelCollectionRDD[94] at parallelize at <console>:30
rdd2 = CoalescedRDD[95] at coalesce at <console>:31
rdd3 = MapPartitionsRDD[99] at repartition at <console>:32


MapPartitionsRDD[99] at repartition at <console>:32

#### 2.1.5.26 repartionAndSortWithinPartitions

- 같은 성격을 지닌 데이터를 같은 파티션으로 분리하고 싶을 때
- RDD를 구성하는 모든 데이터를 특정 기준에 따라서 여러 개의 파티션으로 분리하고 각 파티션 단위로 정렬을 수행한 뒤 이 결과로 새로운 RDD를 생성
- 키와 값 쌍

In [28]:
import org.apache.spark.HashPartitioner

val r = scala.util.Random
val data = for (i <- 1 to 10) yield (r.nextInt(100), "-")
val rdd1 = sc.parallelize(data)
val rdd2 = rdd1.repartitionAndSortWithinPartitions(new HashPartitioner(3))
//결과 검중
rdd2.foreachPartition(it => {
    println("==========")
    it.foreach(v => println(v))
})

r = scala.util.Random$@13a021fc
data = Vector((88,-), (16,-), (17,-), (75,-), (90,-), (41,-), (76,-), (8,-), (75,-), (94,-))
rdd1 = ParallelCollectionRDD[102] at parallelize at <console>:37
rdd2 = ShuffledRDD[103] at repartitionAndSortWithinPartitions at <console>:38


ShuffledRDD[103] at repartitionAndSortWithinPartitions at <console>:38

#### 2.1.5.27 partitionBy

- 키와 값의 쌍
- org.apache.spark.Partitioner 클래스의 인자 

In [30]:
val rdd1 = sc.parallelize(List("apple", "mouse", "monitor"), 5).map{a => (a, a.length)}
val rdd2 = rdd1.partitionBy(new HashPartitioner(2))
println(s"rdd1: ${rdd1.getNumPartitions}, rdd2: ${rdd2.getNumPartitions}")

rdd1: 5, rdd2: 2


rdd1 = MapPartitionsRDD[108] at map at <console>:32
rdd2 = ShuffledRDD[109] at partitionBy at <console>:33


ShuffledRDD[109] at partitionBy at <console>:33

###  필터와 정렬 연산

#### 2.1.5.28 filter

- 원하는 요소만 남기고 원하지 않는 요소는 걸러내는 동작

In [31]:
val rdd = sc.parallelize(1 to 5)
val result = rdd.filter(_ > 2)
println(result.collect.mkString(","))

3,4,5


rdd = ParallelCollectionRDD[110] at parallelize at <console>:32
result = MapPartitionsRDD[111] at filter at <console>:33


MapPartitionsRDD[111] at filter at <console>:33

#### 2.1.5.29 sortByKey

- 키 값을 기준으로 요소를 정렬

In [33]:
val rdd = sc.parallelize(List("q", "z", "a"))
val result = rdd.map((_, 1)).sortByKey()
println(result.collect.mkString(","))    

(a,1),(q,1),(z,1)


rdd = ParallelCollectionRDD[112] at parallelize at <console>:32
result = ShuffledRDD[116] at sortByKey at <console>:33


ShuffledRDD[116] at sortByKey at <console>:33

#### 2.1.5.30 keys, values

- 키와 값의 쌍
- keys()
    - 키에 해당하는 요소로 구성된 RDD 생성
- values()
    - 값에 해당하는 요소로 구성된 RDD 생성

In [34]:
val rdd1 = sc.parallelize(List("apple", "mouse", "monitor"), 5).map{a => (a, a.length)}
println(rdd1.keys.collect.mkString(","))
println(rdd1.values.collect.mkString(","))

apple,mouse,monitor
5,5,7


rdd1 = MapPartitionsRDD[118] at map at <console>:31


MapPartitionsRDD[118] at map at <console>:31

#### 2.1.5.31 sample

- 샘플을 추출해 새로운 RDD 생성

In [36]:
val rdd = sc.parallelize(1 to 100)
val result1 = rdd.sample(false, 0.5)
val result2 = rdd.sample(true, 1.5)
println(result1.take(5).mkString(","))
println(result2.take(5).mkString(","))

3,6,9,13,14
1,3,4,4,4


rdd = ParallelCollectionRDD[124] at parallelize at <console>:33
result1 = PartitionwiseSampledRDD[125] at sample at <console>:34
result2 = PartitionwiseSampledRDD[126] at sample at <console>:35


PartitionwiseSampledRDD[126] at sample at <console>:35

## 2.1.6 RDD 액션
- lazy eveluation
- 계산에 필요한 정보를 누적해서 내포하고 있다가 실제로 계산이 필요한 시점이 되어서야 실행

### 출력과 관련된 연산

#### 2.1.6.1 first
- 첫번째 요소 하나

In [37]:
val rdd = sc.parallelize(List(5, 4, 1))
val result = rdd.first
println(result)

5


rdd = ParallelCollectionRDD[127] at parallelize at <console>:32
result = 5


5

#### 2.1.6.2 take

- 첫번째 요소로부터 순서대로 n개를 추출
- 배열/리스트와 같은 컬렉션 타입 반환

In [38]:
val rdd = sc.parallelize(1 to 20, 5)
val result = rdd.take(5)
println(result.mkString(","))

1,2,3,4,5


rdd = ParallelCollectionRDD[128] at parallelize at <console>:32
result = Array(1, 2, 3, 4, 5)


[1, 2, 3, 4, 5]

#### 2.1.6.3 takeSmple
- 지정된 크기의 샘플을 추출

#### 2.1.6.4 collect, count
- collect()
    - 모든 원소 모음
- count()
    - 원소의 갯수


#### 2.1.6.5 countByValue
- RDD에 속하는 각 값들이 나타나는 횟수를 구해서 맵 행태로 돌려주는 메서드

In [39]:
val rdd = sc.parallelize(List(1,1,2,3,3))
val result = rdd.countByValue
println(result)

Map(1 -> 2, 2 -> 1, 3 -> 2)


rdd = ParallelCollectionRDD[129] at parallelize at <console>:32
result = Map(1 -> 2, 2 -> 1, 3 -> 2)


Map(1 -> 2, 2 -> 1, 3 -> 2)

#### 2.1.6.6 reduce

- RDD에 포함된 임의의 값 두개를 하나로 합치는 함수를 이용해 RDD에 포함된 모든 요소를 하나의 값으로 병합하고 반환하는 메서드

In [40]:
val rdd = sc.parallelize(1 to 10)
val result = rdd.reduce(_ + _)
println(result)

55


rdd = ParallelCollectionRDD[133] at parallelize at <console>:32
result = 55


55

#### 2.1.6.7 fold

- 같은 RDD내의 모든 요소를 대상으로 교환법칙과 결합법칙이 성립되는 바이너리 함수를 순차적용해 최종 결과를 구하는 메서드

In [42]:
val rdd = sc.parallelize(1 to 10)
val result = rdd.fold(0)(_ + _)
println(result)

55


rdd = ParallelCollectionRDD[134] at parallelize at <console>:32
result = 55


55

#### 2.1.6.8 aggregate

- reduce(), fold()는 모두 입력/출력 타입이 동일해야 한다는 제약이 있음
- aggrecate()메서드는 제약이 없음
- 3개의 인자
    - zeroValue: 초기값
    - seqOp : 파티션 내에서 병합
    - combOp : 초종 병합

In [47]:
val rdd = sc.parallelize(List(100, 80, 75, 90, 95), 3)
val zeroValue = Record(0, 0)
val seqOp = (r: Record, v: Int) => r.add(v)
val combOp = (r1: Record, r2: Record) => r1.add(r2)
val result1 = rdd.aggregate(zeroValue)(seqOp, combOp)
println(result1.amount/result1.number)

// 좀 더 간결
val result2 = rdd.aggregate(Record(0, 0))(_ add _, _ add _)
println(result2)

avg: 88


rdd = ParallelCollectionRDD[139] at parallelize at <console>:34
result2 = avg: 88


lastException: Throwable = null


avg: 88

#### 2.1.6.9 sum

- 모든 요소가 double, Long등 숫자 타입일 경우

In [48]:
val rdd = sc.parallelize(1 to 10)
val result = rdd.sum
println(result)

55.0


rdd = ParallelCollectionRDD[140] at parallelize at <console>:32
result = 55.0


55.0

#### 2.1.6.10 foreach, foreachPartition

- foreach()
    - RDD의 모든 요소에 특정함수를 적용하는 메서드
- foreachPartition()
    - 파티션 단위로 적용
    - 실행말 할뿐 리턴하지 않는다.
- 인자로 전달받은 함수가 개별노드에서 실행

In [55]:
val rdd = sc.parallelize(1 to 10, 3)
rdd.foreach{ v => 
    println(s"Value Side Effect: ${v}")
}

rdd.foreachPartition(values => {
    println("Partition Side Effect")
    for(v <- values) println(s"Value Side Effect: ${v}")
})

rdd = ParallelCollectionRDD[143] at parallelize at <console>:31


ParallelCollectionRDD[143] at parallelize at <console>:31

#### 2.1.6.11 toDebugString

- 디버깅을 위한 메서드

In [58]:
val rdd = sc.parallelize(1 to 100, 10).map(_ * 2).persist.map(_ + 1).coalesce(2)
println(rdd.toDebugString)

(2) CoalescedRDD[147] at coalesce at <console>:31 []
 |  MapPartitionsRDD[146] at map at <console>:31 []
 |  MapPartitionsRDD[145] at map at <console>:31 []
 |  ParallelCollectionRDD[144] at parallelize at <console>:31 []


rdd = CoalescedRDD[147] at coalesce at <console>:31


CoalescedRDD[147] at coalesce at <console>:31

#### 2.1.6.12 cache, persist, unpersist

- 액션 연산이 수행될 때마다 관련 트랜스포메이션 연산을 반복
- 기존에 사용했던 데이터가 메모리에 남아 있다면 그 데이터를 사용하지만 다른 이유로 인해 데이터 남아있지 않다면 RDD 생성 히스토리(리니지)를 이용해 복구 수행
- cache와 persist는 첫 액션을 실행한 후에 RDD정보를 메모리 또는 디스크에 저장

In [59]:
import org.apache.spark.storage.StorageLevel

val rdd = sc.parallelize(1 to 100, 10)
rdd.cache
rdd.persist(StorageLevel.MEMORY_ONLY)

rdd = ParallelCollectionRDD[148] at parallelize at <console>:33


ParallelCollectionRDD[148] at parallelize at <console>:33

#### 2.1.6.13 partitions

- RDD의 파티션 정보가 담긴 배열을 리턴

In [61]:
val rdd = sc.parallelize(1 to 100, 10)
println(rdd.partitions.size)
println(rdd.getNumPartitions)

10
10


rdd = ParallelCollectionRDD[149] at parallelize at <console>:32


ParallelCollectionRDD[149] at parallelize at <console>:32

## 2.1.7 RDD 데이터 불러오기와 저장하기

- 다양한 포맷 지원

#### 2.1.7.1 테스트 파일

In [62]:
val rdd = sc.textFile("RDD.ipynb")

rdd = RDD.ipynb MapPartitionsRDD[151] at textFile at <console>:30


RDD.ipynb MapPartitionsRDD[151] at textFile at <console>:30

In [63]:
val rdd = sc.parallelize(1 to 1000, 3)
rdd.saveAsTextFile("./sub1")

rdd = ParallelCollectionRDD[152] at parallelize at <console>:32


ParallelCollectionRDD[152] at parallelize at <console>:32

#### 2.1.7.2 오프젝트 파일(object file)

- object file을 읽고 쓰는 기능 제공

In [64]:
val rdd = sc.parallelize(1 to 1000)
rdd.saveAsObjectFile("./sub_path")
val rdd1 = sc.objectFile[Int]("./sub_path")
println(rdd1.take(10).mkString(","))

126,127,128,129,130,131,132,133,134,135


rdd = ParallelCollectionRDD[154] at parallelize at <console>:34
rdd1 = MapPartitionsRDD[158] at objectFile at <console>:36


MapPartitionsRDD[158] at objectFile at <console>:36

#### 2.1.7.3 시퀀스 파일

- Sequence file은 키와 값으로 구성된 데이터를 저장하는 Binary 파일 포맷

In [65]:
val rdd = sc.parallelize(List("a", "b", "c", "b", "c")).map((_, 1))
rdd.saveAsSequenceFile("data/smaple/saveAsSeqFile/scala")
val rdd2 = sc.sequenceFile[String, Int]("data/smaple/saveAsSeqFile/scala")
println(rdd2.collect.mkString(", "))

(a,1), (b,1), (c,1), (b,1), (c,1)


rdd = MapPartitionsRDD[160] at map at <console>:35
rdd2 = MapPartitionsRDD[163] at sequenceFile at <console>:37


MapPartitionsRDD[163] at sequenceFile at <console>:37

## 2.1.8 클러스터 환경에서의 공유 변수

#### broadcast variable
- 스파크 잡이 실행되는 동안 클러스터 내의 모든 서버에서 공유할 수 있는 일기 전용 자원 설정

In [68]:
val broadcateUsers = sc.broadcast(Set("u1", "u2"))
val rdd = sc.parallelize(List("u1", "u2", "u3", "u4", "u5", "u6"), 3)
val result = rdd.filter(broadcateUsers.value.contains(_))
println(result.collect.mkString(","))

u1,u2


broadcateUsers = Broadcast(93)
rdd = ParallelCollectionRDD[168] at parallelize at <console>:35
result = MapPartitionsRDD[169] at filter at <console>:36


MapPartitionsRDD[169] at filter at <console>:36

#### Accumulator
- 클러스터 내의 모든 서버가 공유하는 쓰기 공간을 제공함으로써 각 서버에서 발생하는 특정 이벤트의 수를 세거나 관찰하고 싶은 정보를 모아두는 등의 용도로 편리하게 활용

In [72]:
val acc1 = sc.longAccumulator("invlidFormat")
val acc2 = sc.collectionAccumulator[String]("invalidFormat2")
val data = List("u1:addr1", "u2:add2", "u3", "u4:addr4")
val rdd = sc.parallelize(data)
rdd.foreach{ v => 
    if (v.split(":").length != 2) {
        acc1.add(1L)
        acc2.add(v)
    }
}
println("잘못된 데이터 수: " + acc1.value)
println("잘못된 데이터: " + acc2.value)

잘못된 데이터 수: 1
잘못된 데이터: [u3]


acc1 = LongAccumulator(id: 16202, name: Some(invlidFormat), value: 1)
acc2 = CollectionAccumulator(id: 16203, name: Some(invalidFormat2), value: [u3])
data = List(u1:addr1, u2:add2, u3, u4:addr4)
rdd = ParallelCollectionRDD[173] at parallelize at <console>:38


ParallelCollectionRDD[173] at parallelize at <console>:38

# 정리

- RDD는 스파크에 다루는 데이터에 대한 추상 모델로서 메모리를 기반으로 동작하면서도 데이터를 처리하는 과정에서 데이터 누락되거나 유실되지 않게 하는 에러 복구 메커니즘이 있음
